In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from itertools import product

import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
import cmocean

import cartopy
import cartopy.crs as ccrs

import xpersist as xp
cache_dir = '/glade/p/cgd/oce/projects/cesm2-marbl/xpersist_cache/3d_fields'
if (os.path.isdir(cache_dir)):
    xp.settings['cache_dir'] = cache_dir
os.makedirs(cache_dir, exist_ok=True)

os.environ['CESMDATAROOT'] = '/glade/scratch/mclong/inputdata'
import pop_tools

import climo_utils as cu
import utils
import calc

import plot

In [3]:
cluster, client = utils.get_ClusterClient()
cluster.scale(12) #adapt(minimum_jobs=0, maximum_jobs=24)
client

Client Scheduler: tcp://10.12.205.24:45593 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
ds_grid = pop_tools.get_grid('POP_gx1v7')

/glade/work/mclong/miniconda3/envs/co2-hole/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 6103. The TBB threading layer is disabled.
  warnings.warn(problem)


In [5]:
def monthly_clim(ds):
    return ds.groupby('time.month').mean('time')

In [6]:
time_slice = slice("1990-01-15", "2015-01-15")
varlist = [
    'TEMP',
    'SALT',    
]
ds_list = []
for variable in varlist:
    xp_func = xp.persist_ds(cu.read_CESM_var, name=f'mld-moc-{variable}', trust_cache=True)    
    ds_list.append(xp_func(
        time_slice, 
        variable, 
        postprocess=monthly_clim,
    ))
    
ds = xr.merge(ds_list)
#dsp = utils.pop_add_cyclic(ds)
#dsp.info()

assuming cache is correct
reading cached file: /glade/p/cgd/oce/projects/cesm2-marbl/xpersist_cache/3d_fields/mld-moc-TEMP.nc
assuming cache is correct
reading cached file: /glade/p/cgd/oce/projects/cesm2-marbl/xpersist_cache/3d_fields/mld-moc-SALT.nc


In [ ]:
xp_func = xp.persist_ds(calc.mld_dsigma, name='historical-mld-moc-MLD_03', trust_cache=True)    
mld_03 = xp_func(ds.SALT, ds.TEMP, ds_grid.KMT, dsigma=0.03)
mld_03

In [ ]:
xp_func = xp.persist_ds(calc.mld_dsigma, name='historical-mld-moc-MLD_125', trust_cache=True)    
mld_125 = xp_func(ds.SALT, ds.TEMP, ds_grid.KMT, dsigma=0.125)
mld_125